Generate sample data

In [1]:
import random

# Lista de posibles valores para cada campo
nombres = ["Juan", "Ana", "Pedro", "Maria", "Carlos", "Sofia"]
apellidos = ["Garcia", "Rodriguez", "Martinez", "Lopez", "Perez", "Gomez"]
tipos_propiedad = ["Casa", "Departamento", "Terreno", "Local comercial"]
cantidades = [100000, 200000, 300000, 400000, 500000]
plazos = [5, 10, 15, 20, 25, 30]


# Función para generar un registro aleatorio
def generar_registro():
    nombre = random.choice(nombres)
    apellido = random.choice(apellidos)
    tipo_propiedad = random.choice(tipos_propiedad)
    cantidad = random.choice(cantidades)
    plazo = random.choice(plazos)
    tasa = round(random.uniform(5, 10), 2)
    return [nombre, apellido, tipo_propiedad, cantidad, plazo, tasa]


# Generar 100 registros
registros = []
for i in range(100):
    registro = generar_registro()
    registros.append(registro)

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Exercise01').getOrCreate()

Ejercicio 1

Crear un DataFrame a partir de los registros y crea una vista temporal

In [4]:
df = spark.createDataFrame(registros, ["nombre", "apellido", "tipo_propiedad", "cantidad", "plazo", "tasa"])
df.createOrReplaceTempView("hipotecas")
df.show()

+------+---------+---------------+--------+-----+----+
|nombre| apellido| tipo_propiedad|cantidad|plazo|tasa|
+------+---------+---------------+--------+-----+----+
|Carlos|    Gomez|Local comercial|  500000|    5|5.39|
| Maria|    Gomez|           Casa|  400000|   20|9.37|
| Sofia| Martinez|           Casa|  300000|   20|5.59|
|   Ana| Martinez|Local comercial|  100000|   15|5.11|
| Pedro|    Gomez|   Departamento|  100000|   25|8.78|
| Pedro|    Lopez|Local comercial|  500000|   20|7.18|
| Sofia|    Lopez|           Casa|  200000|   20|8.53|
| Sofia|    Perez|        Terreno|  400000|   20|5.18|
|Carlos| Martinez|   Departamento|  200000|   30|8.27|
|Carlos|    Lopez|   Departamento|  400000|   30| 7.5|
| Sofia| Martinez|Local comercial|  200000|    5|9.98|
| Pedro|Rodriguez|        Terreno|  300000|   20|6.49|
| Sofia|   Garcia|           Casa|  400000|    5|8.94|
| Sofia|   Garcia|   Departamento|  500000|   25|6.82|
| Pedro|    Gomez|Local comercial|  500000|   30|8.64|
| Sofia|  

Ejercicio 2
Seleccionar los registros de las personas que han tomado una hipoteca mayor de 300.000 €

In [5]:
df.filter(df.cantidad > 300000).show()

+------+---------+---------------+--------+-----+----+
|nombre| apellido| tipo_propiedad|cantidad|plazo|tasa|
+------+---------+---------------+--------+-----+----+
|Carlos|    Gomez|Local comercial|  500000|    5|5.39|
| Maria|    Gomez|           Casa|  400000|   20|9.37|
| Pedro|    Lopez|Local comercial|  500000|   20|7.18|
| Sofia|    Perez|        Terreno|  400000|   20|5.18|
|Carlos|    Lopez|   Departamento|  400000|   30| 7.5|
| Sofia|   Garcia|           Casa|  400000|    5|8.94|
| Sofia|   Garcia|   Departamento|  500000|   25|6.82|
| Pedro|    Gomez|Local comercial|  500000|   30|8.64|
| Pedro| Martinez|   Departamento|  500000|   25|5.15|
|Carlos|Rodriguez|           Casa|  400000|    5| 8.1|
| Pedro|   Garcia|Local comercial|  500000|   30|9.96|
|Carlos|Rodriguez|           Casa|  500000|   10|9.05|
| Sofia|    Perez|   Departamento|  400000|   15|6.54|
| Maria|    Lopez|Local comercial|  500000|   20|9.46|
|Carlos| Martinez|        Terreno|  400000|   10| 5.6|
| Maria|  

Ejercicio 3

Contar cuántos registros hay para cada tipo de propiedad:

In [6]:
df.groupBy("tipo_propiedad").count().show()

+---------------+-----+
| tipo_propiedad|count|
+---------------+-----+
|   Departamento|   26|
|           Casa|   26|
|Local comercial|   28|
|        Terreno|   20|
+---------------+-----+


Ejercicio 4

Obtener la cantidad total de dinero prestado en hipotecas

In [7]:
df.agg({"cantidad": "sum"}).show()

+-------------+
|sum(cantidad)|
+-------------+
|     29400000|
+-------------+


Ejercicio 5

Obtener la tasa promedio de interés para cada plazo

In [8]:
df.groupBy("plazo").agg({"tasa": "avg"}).show()

+-----+-----------------+
|plazo|        avg(tasa)|
+-----+-----------------+
|   25|7.593999999999999|
|    5|7.818571428571429|
|   15|7.560769230769231|
|   20|7.379565217391305|
|   30|7.749411764705881|
|   10|8.063333333333334|
+-----+-----------------+


Ejercicio 6
Seleccionar los registros de las personas que han tomado una hipoteca por un monto mayor a $300,000 y cuyo plazo sea mayor a 15 años

In [9]:
df.filter((df.cantidad > 300000) & (df.plazo > 15)).show()

+------+---------+---------------+--------+-----+----+
|nombre| apellido| tipo_propiedad|cantidad|plazo|tasa|
+------+---------+---------------+--------+-----+----+
| Maria|    Gomez|           Casa|  400000|   20|9.37|
| Pedro|    Lopez|Local comercial|  500000|   20|7.18|
| Sofia|    Perez|        Terreno|  400000|   20|5.18|
|Carlos|    Lopez|   Departamento|  400000|   30| 7.5|
| Sofia|   Garcia|   Departamento|  500000|   25|6.82|
| Pedro|    Gomez|Local comercial|  500000|   30|8.64|
| Pedro| Martinez|   Departamento|  500000|   25|5.15|
| Pedro|   Garcia|Local comercial|  500000|   30|9.96|
| Maria|    Lopez|Local comercial|  500000|   20|9.46|
| Maria|    Perez|        Terreno|  400000|   25|6.07|
|Carlos|    Gomez|        Terreno|  500000|   20|6.78|
| Sofia|   Garcia|   Departamento|  500000|   30|6.66|
|Carlos| Martinez|        Terreno|  400000|   30|7.22|
| Sofia|   Garcia|Local comercial|  500000|   25| 8.4|
|Carlos|   Garcia|        Terreno|  400000|   20|5.05|
| Maria|  